In [1]:
import os
import glob

os.chdir('C:/Users/suzuk/Desktop/PropellerCalc/PropellerTest')
files = glob.glob('*.dat')
print(files)


['PER3_10x10.dat', 'PER3_10x10E.dat', 'PER3_10x3.dat', 'PER3_10x38SF.dat', 'PER3_10x4.dat', 'PER3_10x45MR.dat', 'PER3_10x46SF.dat', 'PER3_10x47SF.dat', 'PER3_10x5.dat', 'PER3_10x55MR.dat', 'PER3_10x5E.dat', 'PER3_10x6-4.dat', 'PER3_10x6.dat', 'PER3_10x6E.dat', 'PER3_10x6F.dat', 'PER3_10x7-3.dat', 'PER3_10x7.dat', 'PER3_10x7E.dat', 'PER3_10x7SF.dat', 'PER3_10x8.dat', 'PER3_10x8E.dat', 'PER3_10x9.dat', 'PER3_11x10.dat', 'PER3_11x10E.dat', 'PER3_11x11.dat', 'PER3_11x12.dat', 'PER3_11x12E.dat', 'PER3_11x12W.dat', 'PER3_11x13.dat', 'PER3_11x14.dat', 'PER3_11x3.dat', 'PER3_11x38SF.dat', 'PER3_11x4.dat', 'PER3_11x45EP.dat', 'PER3_11x45MR.dat', 'PER3_11x46SF.dat', 'PER3_11x47SF.dat', 'PER3_11x5.dat', 'PER3_11x55E.dat', 'PER3_11x55MR.dat', 'PER3_11x6-4.dat', 'PER3_11x6.dat', 'PER3_11x7.dat', 'PER3_11x7E-3.dat', 'PER3_11x7E.dat', 'PER3_11x7SF.dat', 'PER3_11x8.dat', 'PER3_11x85E.dat', 'PER3_11x8E.dat', 'PER3_11x8F.dat', 'PER3_11x9-4.dat', 'PER3_11x9.dat', 'PER3_12x10.dat', 'PER3_12x10E.dat', 'PER

In [5]:
import numpy as np
import pandas as pd
import tkinter as tk
from tkinter import ttk
from tkinter import filedialog
from tkinter import *
from tkinter.ttk import *
import xlsxwriter
import re

#for accessing data sorting data

motorName = 'HackerA40-14L-kV355'

selectAltitude = True         #global variable, False means use Density 

altitude = 0   #in feet
h = altitude/3.281 #convert to meters
Temperature = 15+273.15-6.5*h/1000 #temp in Kelvin
T = Temperature 
density = 1.225

#throttle information
Throttle = 100       #throttle value from 0 to 100

#esc information
eCurrMax = 120     #max current of esc
eRes = 0.004      #esc resistance in ohms

#battery cell information --- specified from Li-Pos
cells = 6           #number of cells in Li-Po battery
battVolt = 3.7      #Li-Po per-cell Voltage
battRes = 0.005    #battery Resistance in ohms

#motor information
kV = 355           #RPM per volt for motor 
nLCurr = 0.85        #no load current in amps
nLVolt = 8.4        #no load voltage in Volts
mRes = 0.050         #motor resistance in ohms
mWatt = 2000          #max watts of motor

velocityStep = 1.524

#'''
n= 1
if n == 1:
    n=0 
    filename = 'PER3_10x4.dat'
#'''
#for filename in files:

    #for reading in file automatically and determining prop diameter + prop pitch
    #---------------------------------------------------------------------#
    firstword = "PER3_"
    divider = "x"

    rx_to_last = r'^.*{}'.format(re.escape(divider))

    #remove PER3_ from filename
    removeBeginning = re.sub(r'^.*?'+firstword, '', filename)

    #remove .dat from filename
    propType = removeBeginning[:len(removeBeginning) - 4]
    #remove paranthesis from propType, messes up regular expressions
    propType = re.sub(r'\([^)]*\)', '', propType)
    
    propPitch = re.sub(r'^.*?'+divider, '', propType)
    
    propDiameter = re.sub(divider+propPitch, '', propType)
    #---------------------------------------------------------------------#
    
    print(filename)   #print filename if having problems

    #set column index to help import data
    header_list =  [str(x) for x in range(12 + 1)]

    #read in file
    propdata = pd.read_csv(filename, sep='\s+', names=header_list)
    propdata = propdata.to_numpy()
    df = pd.DataFrame(propdata)
    df.to_csv(index=False)

    #find all rows with rpm data
    searchString = "RPM"
    mask = np.column_stack([df[col].str.contains(r''+searchString, na=False) for col in df])
    RPMdata = pd.DataFrame.dropna(df.loc[mask.any(axis=1)],axis=1)

    #create new table called RPM values to be able to bin RPM values into groups
    #remove characters from PROP RPM and change to numeric values to make analysis easier
    #first find column with numbers by converting all columns to numbers
    RPMs = RPMdata.apply(pd.to_numeric,errors = 'ignore')

    #then remove all non-int64 columns
    RPMs = RPMs.select_dtypes(include=['int64'])

    #get first RPM value index
    a = RPMs.columns.values           #weird name, maybe switch later, a[] is for locating which column RPM values at
    indexminRPM = RPMs[a[0]].index[0]

    #delete unnecessary top rows of RPM data using first RPM value
    RPMdf = df.iloc[indexminRPM:]

    #get max RPM and index
    maxRPM = RPMs.max()
    indexmaxRPM = RPMs[a[0]].idxmax()

    #get index of each RPM value to bin data
    RPMindex = RPMs.index

    #clear out columns with mostly all NA
    RPMdf = pd.DataFrame.dropna(RPMdf,axis='columns',thresh=10)

    #add index of last row of database to RPM dataframe to bin correctly
    lastIndex = RPMdf.iloc[[-1]].index.values[0]

    #get index of RPMs to group them by their RPM
    RPMindex = np.append(RPMindex,lastIndex)
    RPMlabel = RPMs[a[0]].to_list()

    #label columns as what they're portraying
    #assumes all bins of RPM have same values at same columns
    #assumes V is in first column, find it
    Units = RPMdf[RPMdf[0].str.match('V')]

    #find first row V is in
    unitsRow = Units[0].index[0]

    #take first row and set this as column headers
    RPMdf.columns = RPMdf.loc[unitsRow]

    #remove all non-numeric rows
    RPMdf = RPMdf.apply(lambda x: pd.to_numeric(x, errors='coerce')).dropna()

    #finally group each RPM
    RPMdf = RPMdf.groupby(pd.cut(RPMdf.index, RPMindex, right=False, labels = RPMlabel))

    #for printing out grouped dataframe
    #for name,group in RPMdf:
    #   print(name)
    #   print(group)


    #NOTES#
    #J = Velocity / (RPs*Diameter (m))
    """
    Some Shortened terms

    Curr = Current
    Res = Resistance
    Batt = Battery
    Volt = Voltage
    v = Voltage (when put before a term)
    p = Power (when put before a term)
    Amps = Amperage
    """
    propDiameter = float(propDiameter)    #in inches
    altitude = float(altitude)            #in feet
    density = float(density)              #in kg / m^3
    eCurrMax = float(eCurrMax)            #in amps
    eRes = float(eRes)                    #in ohms
    cells = float(cells)                  #integer
    battVolt = float(battVolt)            #in volts
    battRes = float(battRes)              #in ohms
    kV = float(kV)                        #in RPM/volt
    nLCurr = float(nLCurr)                #in amps
    nLVolt = float(nLVolt)                #in volts
    mRes = float(mRes)                    #in ohms
    mWatt = float(mWatt)                  #in watts
    Throttle = float(Throttle)            #scale from 0 to 100

    """ for debugging
    print(selectAltitude)
    print('propDiameter: '+str(propDiameter))
    print('altitude: '+str(altitude))
    print('density: '+str(density))
    print('eCurrMax: '+str(eCurrMax))
    print('eRes: '+str(eRes))      
    print('battRes: '+str(battRes))           
    print('battVolt: '+str(battVolt))
    print('cells: '+str(cells))
    print('kV: '+str(kV))
    print('nLCurr: '+str(nLCurr))
    #print('nLVolt: '+str(nLVolt))
    print('mRes: '+str(mRes))
    print('mWatt: '+str(mWatt))
    print('Throttle: '+str(Throttle))
    """

    propDiameter = propDiameter*0.0254     #convert prop diameter to meters

    #WARNING --- pressure and altitude calcaulator only valid to approximately 12,000 m
    if(selectAltitude):
        h = altitude/3.281 #convert to meters
        Temperature = 15+273.15-6.5*h/1000 #temp in Kelvin
        T = Temperature 
        R = 8.3143       #constant
        M = 0.02896      #molar mass of Earth's air
        g = 9.806        #gravity
        Po = 101325      #sea level pressure in kPa
        Pressure = Po*np.exp(-M*g*h/(R*T))    #obtained Pressure in kPa
        Density = (Pressure*M)/(R*T)          #unit is in kg/m^3
    else:
        Density = density

    #------------------all these can be forced but are selected by user in above code--------------
    #throttle information force 

    #propDiameter = 0.508  #prop Diameter in meters
    #Throttle = 100       #throttle value from 0 to 100

    #esc information
    #eCurrMax = 160     #max current of esc
    #eRes = 0.001      #esc resistance in ohms

    #battery cell information --- specified from Li-Pos
    #cells = 6           #number of cells in Li-Po battery
    #battVolt = 3.7      #Li-Po per-cell Voltage
    #battRes = 0.012    #battery Resistance in ohms

    #motor information
    #kV = 295            #RPM per volt for motor 
    #nLCurr = 1.7        #no load current in amps
    #nLVolt = 10          #no load voltage in Volts
    #mRes = 0.015         #motor resistance in ohms
    #mCurr = 200         #max current of motor

    #-----------------------------------------------------------------------------------------------

    #Note ---- Prof. Merret at UIUC helped provide these formulas, comments added by me for the most part to try to explain each term

    RPMgroups = np.array(list(RPMdf.groups))
    allAmps = []
    for groups in RPMgroups:
        RPMgroup = RPMdf.get_group(groups)
        PWR = RPMgroup['PWR'].iloc[0]*745.7
        allAmps.append(PWR/(cells*battVolt))
    #allAmps is an array with expected amp draw at max static thrust at each velocity 
    #print(allAmps)
    velocityArray = [0]

    def RPMfinder(motorAmpGuess,battAmpGuess,Output,velocity):

        #main calulation for iteration of pMotorOut
        def pMotorOutCalc(motorAmp,battAmp):

            vBatt = cells*battVolt-cells*battRes*battAmp   #get total battery volts with voltage loss
            vTerm = (vBatt-eRes*battAmp)*Throttle/100      #get voltage drop from esc and throttle value
            vDropMotor = motorAmp*mRes                     #get voltage drop from motor
            pMotorIn = vTerm*battAmp                       #get power motor uses in operation
            vEMF = vTerm-vDropMotor                        #get voltage motor actually has that after esc,battery,throttle losses
            pMotorOut = vEMF*(motorAmp-nLCurr)             #get power motor generates (voltage of motor*(Amps of motor-no load motor Amps))
            pMotorIn = vTerm*(motorAmp-nLCurr)             #get power motor receives

            return pMotorIn,pMotorOut,vTerm,motorAmp,vBatt,battAmp,vEMF

        pMotorIn,pMotorOut,vTerm,motorAmp,vBatt,battAmp,vEMF = pMotorOutCalc(motorAmpGuess,battAmpGuess)

        RPM = vEMF*kV                                  #get revolutions per minute of motor
        RPS = RPM/60                                   #get revolutions per second of motor
        #print('RPS: '+str(RPS))

        '''
        #checks if battery can supply current
        if(battAmp > battVolt/battRes):
            raise Exception("Batteries can't supply this much current: " + str(battAmp))

        #checks if ESC can handle Current
        if(battAmpGuess > eCurrMax):
            raise Exception("ESC will be overloaded, choose larger ESC: " + str(Amps))
        '''   

        #checks if RPM is positive    
        if(RPM < 0):                  
            raise Exception("RPM is negative, something is wrong: " + str(RPM))

        #find closest data to RPM values, both above and below it
        RPMgroups = np.array(list(RPMdf.groups))
        if(RPM > RPMgroups.max()):
            raise Exception('RPM maximum limit reached, no data for higher RPM')
        if(RPM < RPMgroups.min()):
            raise Exception('RPM minimum limit reached, no data for lower RPM')
        bottomRPM = RPMgroups[RPMgroups < RPM].max()
        topRPM = RPMgroups[RPMgroups > RPM].min()

        #then get bottom and top RPM groups
        bottomRPMdata = RPMdf.get_group(bottomRPM).reset_index()
        topRPMdata = RPMdf.get_group(topRPM).reset_index()

        #then get cp and ct of both bottom and top RPM groups for desired velocity
        velocityMPH = velocity*2.23694

        #print('Velocity: ' + str(velocityMPH))
        #print('Index Velocity: ' + str(abs(bottomRPMdata['V']-velocityMPH).idxmin()))
        #print(bottomRPMdata)
        #find relevant indexes
        bottomRPMClosestVelocity = abs(bottomRPMdata['V']-velocityMPH).idxmin()
        topRPMClosestVelocity = abs(topRPMdata['V']-velocityMPH).idxmin()

        #find relevant ct
        bottomct = bottomRPMdata['Ct'].iloc[bottomRPMClosestVelocity]
        topct = topRPMdata['Ct'].iloc[topRPMClosestVelocity]

        #find relevant cp
        bottomcp = bottomRPMdata['Cp'].iloc[bottomRPMClosestVelocity]
        topcp = topRPMdata['Cp'].iloc[topRPMClosestVelocity]

        #find relevant torques
        bottomTorque = bottomRPMdata['Torque'].iloc[bottomRPMClosestVelocity]
        topTorque = topRPMdata['Torque'].iloc[topRPMClosestVelocity]

        '''
        %works for 0 m's
        bottomct = bottomRPMdata['Ct'].iloc[0]
        topct = topRPMdata['Ct'].iloc[0]
        bottomcp = bottomRPMdata['Cp'].iloc[0]
        topcp = topRPMdata['Cp'].iloc[0]
        '''
        '''
        print('bottomct: ' +str(bottomct))
        print('topct: ' +str(topct))
        print('bottomcp: ' +str(bottomcp))
        print('topcp: ' +str(topcp))
        '''
        #interpolate to get cp and ct for motor RPM
        cp = bottomcp+(RPM-bottomRPM)*(topcp-bottomcp)/(topRPM-bottomRPM)
        ct = bottomct+(RPM-bottomRPM)*(topct-bottomct)/(topRPM-bottomRPM)
        torque = bottomTorque+(RPM-bottomRPM)*(topTorque-bottomTorque)/(topRPM-bottomRPM)


        #prop power and thrust calculation
        pProp = Density*RPS**3*propDiameter**5*cp   #get power prop requires in kg*m^2/s^3  (W)
        ThrustProp = Density*RPS**2*propDiameter**4*ct   #get thrust prop generates in kg*m/s^2  (N)

        '''
        print('Density: '+str(Density))
        print('Prop D: ' + str(propDiameter))
        print('Cp: '+str(cp))
        print('\n')
        print('pProp: '+str(pProp))
        print('Thrust Prop: ' + str(ThrustProp))
        print('\n')
        '''
        #in hP and english units
        #pPropEng = pProp*0.74                  #get power prop requires in ft-lb/s
        #pProphp = pProp*0.00134102             #get power prop requires in hp
        #tPropEng = ThrustProp*0.2248090795     #get thrust prop generates in lb

        if Output == 1:
            print('Export final parameters here')

        return pMotorIn,pMotorOut,pProp,vTerm,motorAmp,vBatt,battAmp,ThrustProp,vEMF,RPM,torque

    #first Amp guess
    ImotorGuess = nLCurr+0.01 #make first motor amp draw guess as no load amp draw for motor
    IbattGuess = nLCurr+0.01


    '''
    pMotorIn,pMotorOut,pProp,vTerm,motorAmp,vBatt,battAmp,ThrustProp,vEMF = RPMfinder(ImotorGuess,IbattGuess,0,0)  #run first iteration of RPM finder + pMotorOut

    pESCout = pMotorIn
    vESCin = cells*battVolt-cells*battRes*battAmp
    vESCout = cells*battVolt-(cells*battRes+eRes)*battAmp
    ImotorGuess = pProp/vEMF+nLCurr
    IbattGuess = pESCout/vESCout
    '''
    '''
    print('vESCin: '+str(vESCin))
    print('vESCout: '+str(vESCout))
    print('ImotorGuess: '+str(ImotorGuess))
    print('IbattGuess: '+str(IbattGuess))
    print('Motor Power: ' + str(pMotorOut) + ' W')
    print('Thrust Prop: ' + str(ThrustProp) + ' N')
    print('Prop Power: ' + str(pProp) + ' W')
    print('pMotorIn: ' + str(pMotorIn) + ' W')
    print('break\n')
    ''' 
    #dataframe for storing final results
    resultProp = pd.DataFrame(columns=[ "Battery Current (A)","RPM (rev/min)", "Thrust (N)","Power (W)","Efficiency", "Torque (in-lbf)"])
    resultProp.index.name = 'Velocity (m/s)'


    powerTolerance = 0.01 #how close the power out of the motor should be to the power out of the propeller

    velocityPosition = 0 #velocityArray position
    velocity = velocityArray[velocityPosition] #current velocity in m/s

    #have to run here for the first time to make while loop work
    pMotorIn,pMotorOut,pProp,vTerm,motorAmp,vBatt,battAmp,ThrustProp,vEMF,RPM,Torque = RPMfinder(ImotorGuess,IbattGuess,0,velocity)

    while(ThrustProp>0):

        if(velocity != 0):
            velocityArray.append(velocity)

        timeRun = 0 #variable to determine how often loop has run

        #run first time here to verify while loop
        #print('Velocity: ' + str(velocity))
        pMotorIn,pMotorOut,pProp,vTerm,motorAmp,vBatt,battAmp,ThrustProp,vEMF,RPM,Torque = RPMfinder(ImotorGuess,IbattGuess,0,velocity)

        '''
        print(pProp)
        print(pMotorOut)
        '''

        while(abs(pProp-pMotorOut)>powerTolerance):
            pMotorIn,pMotorOut,pProp,vTerm,motorAmp,vBatt,battAmp,ThrustProp,vEMF,RPM,Torque = RPMfinder(ImotorGuess,IbattGuess,0,velocity)
            '''
            print('Motor Power: ' + str(pMotorOut) + ' W')
            print('Thrust Prop: ' + str(ThrustProp) + ' N')
            print('Prop Power: ' + str(pProp) + ' W')
            print('pMotorIn: ' + str(pMotorIn) + ' W')
            print('prop calc done\n')
            '''
            #checks if motor power out is positive
            if(pMotorOut < 0):
                raise Exception("Motor Power Out is negative, something is wrong")
            pESCOut = pMotorIn
            vESCin = cells*battVolt-cells*battRes*battAmp
            vESCout = cells*battVolt-(cells*battRes+mRes)*battAmp
            ImotorGuess = pProp/vEMF+nLCurr
            IbattGuess = pESCOut/vESCout
            """print('pESC: '+str(pESCOut))
            print('vESCin: '+str(vESCin))
            print('vESCout: '+str(vESCout))
            print('ImotorGuess: '+str(ImotorGuess))
            print('IbattGuess: '+str(IbattGuess))
            """
            timeRun = timeRun+1
            #print('Iteration: ' + str(timeRun) + '     Velocity: '+ str(velocityArray[velocityPosition])+' m\s')

        #run one more time and store results in dataframe
        pMotorIn,pMotorOut,pProp,vTerm,motorAmp,vBatt,battAmp,ThrustProp,vEMF,RPM,Torque = RPMfinder(ImotorGuess,IbattGuess,0,velocity)
        #can keep previous motorAmp and battAmp values as they will be close to new ones most likely

        #print('motorAmp: '+str(motorAmp))
        #print('battAmp: '+str(battAmp))

        efficiency = pMotorOut/pMotorIn

        resultProp.loc[velocityArray[velocityPosition]] = [motorAmp,RPM,ThrustProp,pProp,efficiency,Torque]

        velocityPosition = velocityPosition+1
        velocity = velocity + velocityStep

        #print('New Velocity')

    #add thrust in pounds
    thrustLB = resultProp.loc[:,'Thrust (N)']*0.224809
    resultProp['Thrust (lbf)'] = thrustLB
    
    #drop last row because FN is 0 so results may be invalid
    resultProp.drop(resultProp.tail(1).index,inplace=True)
    resultProp
    
    #create filename
    fileNameNew = motorName+'_Prop'+propType+'.xlsx'
    
    
    
    #define sheet name
    sheet_name = fileNameNew[:len(fileNameNew) - 5]

    writer = pd.ExcelWriter(fileNameNew, engine='xlsxwriter')
    
    #export data to writer
    resultProp.to_excel(fileNameNew, sheet_name=sheet_name)
    
    #rest of this is for graphing
    workbook = writer.book
    worksheet = writer.sheets

   # Create a chart object.
    chart = workbook.add_chart({'type': 'scatter'})

    # Configure the series of the chart from the dataframe data.
    max_row = len(resultProp)
    for i in range(len(cat_2)):
        col = i + 1
        chart.add_series({
            'name':       ['Sheet1', 0, col],
            'categories': ['Sheet1', 1, 0, max_row, 0],
            'values':     ['Sheet1', 1, col, max_row, col],
            'marker':     {'type': 'circle', 'size': 7},
        })

    # Configure the chart axes.
    chart.set_x_axis({'name': 'Index'})
    chart.set_y_axis({'name': 'Data Value',
                      'major_gridlines': {'visible': False}})


    # Insert the chart into the worksheet.
    worksheet.insert_chart('K2', chart)

    # Close the Pandas Excel writer and output the Excel file.
    writer.save()

PER3_10x4.dat


NameError: name 'cat_2' is not defined